### 1. Import libraries

In [ ]:
pip install pmdarima

In [ ]:
# Data Analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Time seres
from statsmodels.tsa.stattools import adfuller, acf, pacf, q_stat
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.statespace.tools import diff

# Model selection/Comparison
from pmdarima import auto_arima
from scipy.stats import boxcox
from scipy.special import inv_boxcox

# Metrics/Eval
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tools.eval_measures import aic, bic
import itertools

### 2. Load and Inspect Data

In [ ]:
# Reading excel file into df
raw_df = pd.read_excel('../data/Metro_Interstate_Traffic_Volume.xlsx')

In [ ]:
# Selecting the variables for ARIMA forecasting
df = raw_df[['traffic_volume', 'date_time']]

In [ ]:
# Check for nulls and data types
df.info()

In [ ]:
# Generate basic statistics for variables
df.describe()

In [ ]:
# Check for duplicates
mask = df.duplicated(subset=['date_time'], keep=False)
dfmask = df[mask]

In [ ]:
# Show index of duplicates
dfmask.head(20)

In [ ]:
df[170:300]

In [ ]:
# Remove duplicates
df = df.drop_duplicates(keep='first')

In [ ]:
# Around 8k duplicates removed
df.info()

### 3. Exploratory Plots

In [ ]:
# Plot traffic volume against time(year)
plt.title("Traffic Volume over time(year)")
plt.scatter(df.date_time, df.traffic_volume, alpha=0.5, s=1)
plt.xlabel('Time')
plt.ylabel('Traffic Volume')
plt.show()

We must take note of the missing data between the dates: 2014-08-08 01:00:00 to 2015-06-11 20:00:00

10 months of missing data will impact the precision of our forecast. We may have to interpolate the missing months with the cost of added synthetic variance that may distort long-term forecasting or cut out and do our analysis on the longest continous block of data.

Using the more recent continuous block of data (starting mid 2016) could be more informative for short-midterm forecasting and ensures more reliability within that scope.

In [ ]:
# Time must be the index for ARIMA modeling
df = df.set_index('date_time')

In [ ]:
df.head()

In [ ]:
# Shows annual seasonality
df['month'] = df.index.month
hourly_avg = df.groupby('month')['traffic_volume'].mean()
hourly_avg.plot(kind='line', figsize=[8, 4], title='AVG traffic volume by Month')
plt.xlabel('Monthly')
plt.ylabel('Avg traffic volume')
plt.show()

In [ ]:
# Shows variation cycle of traffic in a month
df['daily'] = df.index.day
hourly_avg = df.groupby('daily')['traffic_volume'].mean()
hourly_avg.plot(kind='line', figsize=[8, 4], title='AVG traffic volume by Day of Month')
plt.xlabel('Day of month')
plt.ylabel('Avg traffic volume')
plt.show()

In [ ]:
# Daily Rush Hour Cycle
df['hour'] = df.index.hour
hourly_avg = df.groupby('hour')['traffic_volume'].mean()
hourly_avg.plot(kind='line', figsize=[8, 4], title='AVG traffic volume by Hour')
plt.xlabel('Hour of day')
plt.ylabel('Avg traffic volume')
plt.show()

### 4. Preprocessing

In [ ]:
# Find the months of discontinuity in the data
df.head()

In [ ]:
# Use the continuous block of data
df_clean = df.loc['2015-06-11 20:00:00':]

In [ ]:
# Around half of the data points were cut, we will focu our forecasting on hourly/daily
df_clean.info()

In [ ]:
df_clean = df_clean.asfreq('H')
df_clean['traffic_volume'].isna().sum()

In [ ]:
df_clean['missing'] = df_clean['traffic_volume'].isna()
df_clean['gap'] = df_clean['missing'].astype(int).diff().ne(0).cumsum()
gap_lengths = df_clean.groupby('gap')['missing'].sum()
gap_lengths[gap_lengths > 0].sort_values(ascending=False)

In [ ]:
# Identify and get rid of gaps longer than 24 hours
long_gaps = gap_lengths[gap_lengths > 24].index

for g in long_gaps:
    df_clean.loc[df_clean['gap'] == g, 'traffic_volume'] = np.nan

# interpolate short gaps
df_clean['traffic_volume'] = df_clean['traffic_volume'].interpolate(limit=12, limit_direction='both')

In [ ]:
df_clean.head()

In [ ]:
# Get rid of EDA variables to prepare for modeling
df_clean = df_clean[['traffic_volume']]

In [ ]:
df_clean.info()

In [ ]:
df_clean.isna().sum()

In [ ]:
# discard nulls that were not interpolated within the 12 hour limit
df_clean = df_clean.dropna(subset='traffic_volume')

In [ ]:
# check all null values were dealt with
df_clean.isna().sum()

In [ ]:
# Confirm we have only an hourly frequency
df_clean.index.diff().value_counts().head()

In [ ]:
# Reinforce Frequency and interpolate the few remaining points
df_clean = df_clean.asfreq('H')
df_clean['traffic_volume'] = df_clean['traffic_volume'].interpolate(method='time')

In [ ]:
# Check interpolation worked
df_clean.index.diff().value_counts().head()

In [ ]:
# Basic Stats check
df_clean.describe()

In [ ]:
# Check for outliers
df_clean['traffic_volume'].plot.box()

### 5. Check Stationarity

### 6. ACF/PACF Analysis

### 7. Fit ARIMA Model

### 8. Forecast

### 9. Evaluation